In [1]:
import numpy as np
import pandas as pd
#import matplotlib as plt
#import seaborn as sns
import os

In [2]:
df = pd.read_csv('fashion_data_with_ratings.csv')

In [3]:
df.head()

,Unnamed: 0,ProductId,Gender,Category,SubCategory,ProductType,Colour,Usage,ProductTitle,Image,ImageURL,rating
0,0,42419,Girls,Apparel,Topwear,Tops,White,Casual,Gini and Jony Girls Knit White Top,42419.jpg,http://assets.myntassets.com/v1/images/style/p...,2.0
1,1,34009,Girls,Apparel,Topwear,Tops,Black,Casual,Gini and Jony Girls Black Top,34009.jpg,http://assets.myntassets.com/v1/images/style/p...,1.5
2,2,40143,Girls,Apparel,Topwear,Tops,Blue,Casual,Gini and Jony Girls Pretty Blossom Blue Top,40143.jpg,http://assets.myntassets.com/v1/images/style/p...,3.5
3,3,23623,Girls,Apparel,Topwear,Tops,Pink,Casual,Doodle Kids Girls Pink I love Shopping Top,23623.jpg,http://assets.myntassets.com/v1/images/style/p...,4.0
4,4,47154,Girls,Apparel,Bottomwear,Capris,Black,Casual,Gini and Jony Girls Black Capris,47154.jpg,http://assets.myntassets.com/v1/images/style/p...,2.5


In [4]:
df.shape

(2906, 12)

In [5]:
df.isnull().sum()

Unnamed: 0      0
ProductId       0
Gender          0
Category        0
SubCategory     0
ProductType     0
Colour          0
Usage           0
ProductTitle    0
Image           0
ImageURL        0
rating          0
dtype: int64

In [6]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [7]:
#df['Category'] = df['Category'].apply(collapse)
#df['SubCategory'] = df['SubCategory'].apply(collapse)
#df['ProductType'] = df['ProductType'].apply(collapse)
#df['Colour'] = df['Colour'].apply(collapse)
#df['Usage'] = df['Usage'].apply(collapse)

In [8]:
df['tags'] = df['Category'] + df['SubCategory'] + df['ProductType'] + df['Colour'] + df['Usage']

In [9]:
df.head()

,Unnamed: 0,ProductId,Gender,Category,SubCategory,ProductType,Colour,Usage,ProductTitle,Image,ImageURL,rating,tags
0,0,42419,Girls,Apparel,Topwear,Tops,White,Casual,Gini and Jony Girls Knit White Top,42419.jpg,http://assets.myntassets.com/v1/images/style/p...,2.0,ApparelTopwearTopsWhiteCasual
1,1,34009,Girls,Apparel,Topwear,Tops,Black,Casual,Gini and Jony Girls Black Top,34009.jpg,http://assets.myntassets.com/v1/images/style/p...,1.5,ApparelTopwearTopsBlackCasual
2,2,40143,Girls,Apparel,Topwear,Tops,Blue,Casual,Gini and Jony Girls Pretty Blossom Blue Top,40143.jpg,http://assets.myntassets.com/v1/images/style/p...,3.5,ApparelTopwearTopsBlueCasual
3,3,23623,Girls,Apparel,Topwear,Tops,Pink,Casual,Doodle Kids Girls Pink I love Shopping Top,23623.jpg,http://assets.myntassets.com/v1/images/style/p...,4.0,ApparelTopwearTopsPinkCasual
4,4,47154,Girls,Apparel,Bottomwear,Capris,Black,Casual,Gini and Jony Girls Black Capris,47154.jpg,http://assets.myntassets.com/v1/images/style/p...,2.5,ApparelBottomwearCaprisBlackCasual


In [10]:
df1 = df.drop(columns=['Image','ImageURL'])

In [11]:
#df2=df1
#df2['tags'] = df1['tags'].apply(lambda x:x.split())

In [12]:
#df2['tags'] = df2['tags'].apply(lambda x: " ".join(x))

In [13]:
df1.head(2)

,Unnamed: 0,ProductId,Gender,Category,SubCategory,ProductType,Colour,Usage,ProductTitle,rating,tags
0,0,42419,Girls,Apparel,Topwear,Tops,White,Casual,Gini and Jony Girls Knit White Top,2.0,ApparelTopwearTopsWhiteCasual
1,1,34009,Girls,Apparel,Topwear,Tops,Black,Casual,Gini and Jony Girls Black Top,1.5,ApparelTopwearTopsBlackCasual


In [14]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [15]:
vector = cv.fit_transform(df1['tags']).toarray()

In [16]:
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [17]:
vector.shape

(2906, 326)

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
similarity = cosine_similarity(vector)
similarity

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [20]:
def recommend(movie):
    index = df1[df1['ProductTitle'] == movie].index[0]
#     print(index)
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
#     print(distances)
    for i in distances[1:6]:
#         print(i)
        print(df1.iloc[i[0]])
        print(df1.iloc[i[0]].ProductTitle)
        

In [21]:
recommend('Gini and Jony Girls Black Top')
# recommend('Do u speak Green Girls Olive 3/4 Pant')
# recommend('Nike Men Zoom Hyperenforcer White Sports Shoes')

Unnamed: 0                                 58
ProductId                               31121
Gender                                  Girls
Category                              Apparel
SubCategory                           Topwear
ProductType                              Tops
Colour                                  Black
Usage                                  Casual
ProductTitle    Gini and Jony Girls Black Top
rating                                    2.0
tags            ApparelTopwearTopsBlackCasual
Name: 58, dtype: object
Gini and Jony Girls Black Top
Unnamed: 0                                    108
ProductId                                   18178
Gender                                      Girls
Category                                  Apparel
SubCategory                               Topwear
ProductType                                  Tops
Colour                                      Black
Usage                                      Casual
ProductTitle    Doodle Kids Girls Black 

In [22]:
import pickle

In [23]:
pickle.dump(df1,open('Products.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))